# Convolutional Neural Networks (CNN)

![Figure 21.4](images/fig_21_4.png)

In [1]:
conv_w = [[[1., -1., 1.]]] # out_channels: 1, in_channels: 1, kernel_size: 3
conv_b = [0.] # out_channels: 1
in_arr = [[[5., 6., 6., 2., 5., 6., 5.]]] # batch size: 1, num channel: 1, signal length: 7

In [2]:
import numpy as np
import tensorflow as tf
import torch

2023-05-09 22:18:32.353946: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-09 22:18:32.356219: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-09 22:18:32.389722: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-09 22:18:32.390461: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-09 22:18:32.879330: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

## PyTorch

### Prepare the tensors

In [3]:
conv_w_t = torch.Tensor(conv_w)
conv_b_t = torch.Tensor(conv_b)
in_t = torch.Tensor(in_arr)

### The implementation in PyTorch

#### Method \#1: Using the graph building block

In [4]:
# create an instance of the 1-D convolution module
conv = torch.nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3, stride=2)
# assign the weights to match Figure 21.4
for name, param in conv.named_parameters():
    if name == "weight":
        param.data = torch.nn.parameter.Parameter(conv_w_t)
    elif name == "bias":
        param.data = torch.nn.parameter.Parameter(conv_b_t)
    else:
        raise NotImplementedError
    print(name, param, "\n")
# forward-pass the input
with torch.no_grad():
    out_t = conv(in_t)
print("Output: ", out_t)

weight Parameter containing:
tensor([[[ 1., -1.,  1.]]], requires_grad=True) 

bias Parameter containing:
tensor([0.], requires_grad=True) 

Output:  tensor([[[5., 9., 4.]]])


#### Method \#2: Using the functional interface

In [5]:
# do 1-D convolution on the input
# the weights of the 1-D convolution function is matched to Figure 21.4
with torch.no_grad():
    out_t = torch.nn.functional.conv1d(input=in_t, weight=conv_w_t, bias=conv_b_t, stride=2)
print("Output: ", out_t)

Output:  tensor([[[5., 9., 4.]]])


## TensorFlow

### Prepare the tensors

In [6]:
# by default, TF operations assume the tensors are channel-last
# so, we transpose our input from (batch size, num channel, signal length) to (batch size, signal length, num channel)
in_t = tf.transpose(tf.constant(in_arr), [0, 2, 1])

2023-05-09 22:18:35.453604: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-09 22:18:35.453778: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


### The implementation in TensorFlow

In [7]:
# create an instance of the 1-D convolution module
conv = tf.keras.layers.Conv1D(filters=1, kernel_size=3, strides=2, input_shape=(None, 1))
_ = conv(in_t)  # initialize the weights before setting the weights
# assign the weights to match Figure 21.4
conv.set_weights([
    np.array(conv_w).T,  # need to transpose because of the channel-last in TF operations
    np.array(conv_b)
])
print("weight and bias: ", conv.get_weights(), "\n")
# forward-pass the input
outputs = conv(in_t)
print("Output: ", outputs)

weight and bias:  [array([[[ 1.]],

       [[-1.]],

       [[ 1.]]], dtype=float32), array([0.], dtype=float32)] 

Output:  tf.Tensor(
[[[5.]
  [9.]
  [4.]]], shape=(1, 3, 1), dtype=float32)
